In [12]:
# import os
# import pandas as pd

# from pydantic import SecretStr
# from langchain.docstore.document import Document
# from langchain.vectorstores import Chroma
# from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from langchain_google_genai import ChatGoogleGenerativeAI

# from langchain.chains import RetrievalQA

# from environments import GEMINI_API_KEY
# from prompts.rentabilidad_agricola import agricola_custom_prompt

# from langchain.agents import initialize_agent
# from langchain.schema import SystemMessage, HumanMessage
# from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain.agents import Tool
# from langchain.agents import AgentExecutor
# from langchain.agents import create_structured_chat_agent
# from langchain.prompts import ChatPromptTemplate
# from langchain.memory import ConversationBufferMemory

# import os
# import pandas as pd
# import re
# from pydantic import SecretStr
# from langchain_core.documents import Document
# from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
# from langchain_community.vectorstores import Chroma
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.prompts import PromptTemplate
# from langchain.chains import RetrievalQA
# from langchain.agents import initialize_agent, Tool
# from langchain.agents.agent_types import AgentType
# from langchain.tools import StructuredTool, Tool
# from typing import Annotated

# from langchain.agents.format_scratchpad import format_to_openai_functions

# Básicos
import os
import pandas as pd
import re
import textwrap
from typing import Annotated
from pydantic import SecretStr

# LangChain Google
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

# Vector store y documentos
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

# RAG y prompts
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Agent + Tools nuevos (✅ correctos)
from langchain.tools import StructuredTool, Tool
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

# Otros (si los usas)
from langchain.memory import ConversationBufferMemory
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.prompts import ChatPromptTemplate

# Custom
from environments import GEMINI_API_KEY
from prompts.rentabilidad_agricola import agricola_custom_prompt

from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage


In [2]:
# Validación clave
if not GEMINI_API_KEY:
    raise ValueError("Gemini API Key not found")

gemini_api_secret = SecretStr(GEMINI_API_KEY)

In [5]:
# Cargar el dataset
df = pd.read_csv("docs/datos_agricolas_3000_registros.csv")
df["Fecha"] = pd.to_datetime(df["Fecha"])

# Crear texto por fila
documentos = []
for _, row in df.iterrows():
    texto = f"""
    Fecha: {row['Fecha']}, Temporada: {row['Temporada']}, Cultivo: {row['Cultivo']}, Parcela: {row['Parcela']},
    Fase Productiva: {row['Fase Productiva']}, Cantidad (kg): {row['Cantidad (kg)']}, Precio por kg: {row['Precio por kg']},
    Ingresos: {row['Ingresos generados']}, Costos: {row['Costo Total']}, Rentabilidad: {row['Rentabilidad']},
    Clima: {row['Clima']}, Demanda de Mercado: {row['Demanda de Mercado']},
    Proyección Precio Temporada Siguiente: {row['Proyección Precio Temporada Siguiente']},
    Observaciones: {row['Observaciones']}, Línea Productiva: {row['Línea Productiva']}
    """.strip()
    documentos.append(texto)

# Convertir a Documentos de LangChain
docs = [Document(page_content=texto) for texto in documentos]

# Embeddings con Gemini
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=gemini_api_secret.get_secret_value())

# Persistencia de vectores
persist_dir = "./chroma_rentabilidad_gemini"

if os.path.exists(persist_dir) and os.path.isdir(persist_dir):
    vectorstore = Chroma(persist_directory=persist_dir, embedding_function=embeddings)
else:
    vectorstore = Chroma.from_documents(docs, embeddings, persist_directory=persist_dir)
    vectorstore.persist()



/tmp/ipykernel_2817/1629674001.py:28: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory=persist_dir, embedding_function=embeddings)


In [15]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

# Chain de Pregunta-Respuesta con Gemini
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatGoogleGenerativeAI(
        model = "models/gemini-1.5-flash-latest",
        google_api_key=gemini_api_secret.get_secret_value(),
        temperature=0.8,
        max_tokens=500
    ),
    retriever=retriever,
    return_source_documents=True
)

In [17]:
pregunta = "dame la mayor rentabilidad de la parcela A en el 2023"
respuesta = qa_chain({"query": pregunta})
partido = textwrap.wrap(respuesta["result"], width=100)
print("\n".join(partido))

La información proporcionada muestra únicamente pérdidas (rentabilidad negativa) para la Parcela A
en 2023.  La mayor rentabilidad, por lo tanto, es la pérdida menor, que fue de -184.0 (en la fecha
2023-03-26, pero esta corresponde a la Parcela D, no la A).  Para la Parcela A, la menor pérdida fue
de -241.0.


In [5]:
# Hacer pregunta
pregunta = "¿Cuál fue la parcela de mayor ingresos en el 2023?"
respuesta = qa_chain({"query": pregunta})
respuesta

{'query': '¿Cuál fue la parcela de mayor ingresos en el 2023?',
 'result': 'Basado en los datos proporcionados, la Parcela D tuvo los mayores ingresos en el 2023, con un total de 893.52.',
 'source_documents': [Document(metadata={}, page_content='Fecha: 2023-12-22 00:00:00, Temporada: 2023-Invierno, Cultivo: Trigo, Parcela: Parcela D,\n    Fase Productiva: Cosecha, Cantidad (kg): 646, Precio por kg: 1.01,\n    Ingresos: 652.46, Costos: 348, Rentabilidad: 304.46,\n    Clima: Lluvioso, Demanda de Mercado: Baja,\n    Proyección Precio Temporada Siguiente: 0.95,\n    Observaciones: Actividad de cosecha bajo clima lluvioso, relacionado con parcela Parcela D., Línea Productiva: Grano'),
  Document(metadata={}, page_content='Fecha: 2023-12-16 00:00:00, Temporada: 2023-Invierno, Cultivo: Trigo, Parcela: Parcela A,\n    Fase Productiva: Cosecha, Cantidad (kg): 352, Precio por kg: 0.94,\n    Ingresos: 330.88, Costos: 225, Rentabilidad: 105.88,\n    Clima: Lluvioso, Demanda de Mercado: Baja,\n   

In [6]:
pregunta = "¿Cuál fue la rentabilidad total de la Parcela A en 2023?"
respuesta = qa_chain({"query": pregunta})
respuesta

{'query': '¿Cuál fue la rentabilidad total de la Parcela A en 2023?',
 'result': 'Para calcular la rentabilidad total de la Parcela A en 2023, se deben sumar las rentabilidades de cada registro: -178.0 + (-469.0) + (-389.0) + (-186.0) + (-291.0) + (-179.0) + (-170.0) + (-420.0) + (-479.0) + (-453.0) = -3214.0\n\nLa rentabilidad total de la Parcela A en 2023 fue de **-3214.0**.',
 'source_documents': [Document(metadata={}, page_content='Fecha: 2023-12-24 00:00:00, Temporada: 2023-Invierno, Cultivo: Café, Parcela: Parcela A,\n    Fase Productiva: Mantenimiento, Cantidad (kg): 0, Precio por kg: 0.0,\n    Ingresos: 0.0, Costos: 178, Rentabilidad: -178.0,\n    Clima: Nublado, Demanda de Mercado: Media,\n    Proyección Precio Temporada Siguiente: 5.1,\n    Observaciones: Actividad de mantenimiento bajo clima nublado, relacionado con parcela Parcela A., Línea Productiva: Tropicales'),
  Document(metadata={}, page_content='Fecha: 2023-07-04 00:00:00, Temporada: 2023-Invierno, Cultivo: Café, Pa

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", google_api_key=GEMINI_API_KEY, temperature=0.2)

# Tool 1: Rentabilidad por parcela
def rentabilidad_por_parcela_func(input_str: str) -> str:
    """
    Calcula la rentabilidad total de una parcela en un año específico.
    Ejemplo válido de input: "Parcela A, 2023"
    """
    match = re.search(r"([Pp]arcela\s*[A-Za-z])[,;]?\s*(\d{4})", input_str)
    if not match:
        return "Entrada inválida. Usa el formato: 'Parcela A, 2023'"

    parcela = match.group(1).strip()
    anio = int(match.group(2))

    df = pd.read_csv("docs/datos_agricolas_3000_registros.csv")
    df["Fecha"] = pd.to_datetime(df["Fecha"])

    df_filtrado = df[(df["Parcela"] == parcela) & (df["Fecha"].dt.year == anio)]

    if df_filtrado.empty:
        return f"No se encontraron datos para la parcela {parcela} en el año {anio}."

    total = df_filtrado["Rentabilidad"].sum()
    return f"La rentabilidad total de la parcela {parcela} en {anio} fue de {total:.2f}"

rentabilidad_tool = Tool(
    name="rentabilidad_por_parcela",
    func=rentabilidad_por_parcela_func,
    description="Calcula la rentabilidad total de una parcela en un año. Formato: 'Parcela A, 2023'"
)

# Tool 2: RAG de contexto agrícola
retriever_tool = Tool(
    name="consultar_datos_agricolas",
    func=lambda q: qa_chain.invoke({"query": q})["result"],
    description="Usa esto para buscar información sobre cultivo, clima, observaciones, demanda de mercado, rentabilidad y más."
)

# ---------------------------------------------
# 5. Agente con Gemini + Tools
# ---------------------------------------------
tools = [rentabilidad_tool, retriever_tool]

agent = initialize_agent(
    tools=tools,
    llm=ChatGoogleGenerativeAI(
        model="gemini-1.5-flash-latest",
        google_api_key=gemini_api_secret.get_secret_value(),
        temperature=0.2
    ),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


/tmp/ipykernel_269294/1399656310.py:45: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [ ]:
respuesta = agent.invoke("¿Cuál fue la rentabilidad de la parcela A en 2023?")
print(respuesta)



> Entering new AgentExecutor chain...
Thought: Necesito usar la función `rentabilidad_por_parcela` para obtener la rentabilidad de la parcela A en 2023.

Action: rentabilidad_por_parcela
Action Input: 'Parcela A, 2023'
Observation: ✅ La rentabilidad total de la parcela Parcela A en 2023 fue de 6776.89
Thought:Thought: I now know the final answer
Final Answer: La rentabilidad de la parcela A en 2023 fue de 6776.89.

> Finished chain.
🧠 {'input': '¿Cuál fue la rentabilidad de la parcela A en 2023?', 'output': 'La rentabilidad de la parcela A en 2023 fue de 6776.89.'}


# Composicion del Prompt con Markdown

In [12]:
qa_template = """
Basado en el siguiente contexto, responde a la pregunta de manera precisa:

Contexto:
{context}

Pregunta:
{question}

Respuesta:
"""

qa_prompt = PromptTemplate(
    template=qa_template,
    input_variables=["context", "question"]
)

qa_chain = RetrievalQA.from_chain_type(
    llm=ChatGoogleGenerativeAI(
        model="gemini-1.5-flash-latest",
        google_api_key=gemini_api_secret.get_secret_value(),
        temperature=0.2
    ),
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": qa_prompt}
)

# ---------------------------------------------
# 4. Definición de Tools
# ---------------------------------------------

def rentabilidad_por_parcela(parcela: str, anio: int) -> str:
    """Calcula la rentabilidad total de una parcela en un año específico."""
    df = pd.read_csv("docs/datos_agricolas_3000_registros.csv")
    df["Fecha"] = pd.to_datetime(df["Fecha"])
    if not parcela.lower().startswith("parcela"):
        parcela = f"Parcela {parcela.upper()}"
    df_filtrado = df[(df["Parcela"] == parcela) & (df["Fecha"].dt.year == anio)]
    if df_filtrado.empty:
        return f"No se encontraron datos para la parcela {parcela} en el año {anio}."
    total = df_filtrado["Rentabilidad"].sum()
    return f"La rentabilidad total de la parcela {parcela} en {anio} fue de ${total:.2f}"

def porcentaje_costos_por_fase() -> str:
    """Retorna un análisis porcentual de los costos según las fases productivas."""
    df = pd.read_csv("docs/datos_agricolas_3000_registros.csv")
    costos_por_fase = df.groupby("Fase Productiva")["Costo Total"].sum()
    total_costos = costos_por_fase.sum()
    porcentajes = (costos_por_fase / total_costos * 100).round(2)
    resultado = "📊 Análisis porcentual de los costos por fase productiva:\n\n"
    for fase, porcentaje in porcentajes.items():
        resultado += f"• {fase}: {porcentaje:.2f}% del total de costos\n"
    return resultado or "No se pudo calcular un resultado en este momento."


def analisis_demanda_mercado() -> str:
    """Relación entre demanda, rentabilidad, precio y volumen."""
    df = pd.read_csv("docs/datos_agricolas_3000_registros.csv")
    df = df.dropna(subset=["Demanda de Mercado", "Rentabilidad", "Precio por kg", "Cantidad (kg)"])
    resumen = df.groupby("Demanda de Mercado").agg({
        "Rentabilidad": "mean",
        "Precio por kg": "mean",
        "Cantidad (kg)": "sum"
    }).round(2)
    resultado = "📈 Análisis de la demanda de mercado:\n\n"
    for demanda, row in resumen.iterrows():
        resultado += (
            f"• Demanda **{demanda}**:\n"
            f"   - Rentabilidad promedio: ${row['Rentabilidad']:.2f}\n"
            f"   - Precio promedio por kg: ${row['Precio por kg']:.2f}\n"
            f"   - Volumen total: {int(row['Cantidad (kg)'])} kg\n\n"
        )
    return resultado or "No se pudo calcular un resultado en este momento."

def consultar_datos_agricolas(query: str) -> str:
    """Consulta semántica al contexto vectorizado."""
    result = qa_chain.invoke({"query": query})
    if not result["result"].strip():
        return "No se encontró información relevante en el contexto para responder la pregunta."
    return result["result"]


# ---------------------------------------------
# 5. Armar Tools con LangChain
# ---------------------------------------------
tools = [
    StructuredTool.from_function(
        func=rentabilidad_por_parcela,
        name="rentabilidad_por_parcela",
        description="Calcula la rentabilidad total de una parcela. Requiere los parámetros: parcela (str) y anio (int)."
    ),
    Tool.from_function(
        func=porcentaje_costos_por_fase,
        name="porcentaje_costos_por_fase",
        description="Retorna el porcentaje de costos por cada fase productiva."
    ),
    Tool.from_function(
        func=analisis_demanda_mercado,
        name="analisis_demanda_mercado",
        description="Muestra relación entre demanda de mercado, rentabilidad y precio."
    ),
    Tool.from_function(
        func=consultar_datos_agricolas,
        name="consultar_datos_agricolas",
        description="Busca en el contexto vectorizado información sobre cultivos, clima, observaciones, etc."
    )
]

# ---------------------------------------------
# 6. Prompt de sistema (custom_markdown_template)
# ---------------------------------------------
custom_markdown_template = """
Eres un asistente agrícola financiero experto en analizar cultivos, rentabilidad, fases productivas, clima y demanda de mercado.
Responde en markdown con viñetas claras y conclusiones útiles para la toma de decisiones.
"""

# ---------------------------------------------
# 7. Inicializar agente
# ---------------------------------------------
agent = initialize_agent(
    tools=tools,
    llm=ChatGoogleGenerativeAI(
        model="gemini-1.5-flash-latest",
        google_api_key=gemini_api_secret.get_secret_value(),
        temperature=0.2
    ),
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

# ---------------------------------------------
# 8. Ejemplo de ejecución
# ---------------------------------------------
respuesta = agent.invoke("¿Cuál fue la rentabilidad de la parcela B en 2023?")
print("🧠", respuesta)



> Entering new AgentExecutor chain...

Invoking: `rentabilidad_por_parcela` with `{'parcela': 'B', 'anio': 2023.0}`


La rentabilidad total de la parcela Parcela B en 2023 fue de $-14034.90

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 * GenerateContentRequest.contents[2].parts: contents.parts must not be empty.


In [23]:
# ---------------------------------------------
# 4. Herramientas
# ---------------------------------------------
def rentabilidad_por_parcela(
    parcela: Annotated[str, "Nombre de la parcela, ejemplo 'Parcela A' o 'B'"],
    anio: Annotated[int, "Año de consulta"]
) -> str:
    df = pd.read_csv("docs/datos_agricolas_3000_registros.csv")
    df["Fecha"] = pd.to_datetime(df["Fecha"])
    if not parcela.lower().startswith("parcela"):
        parcela = f"Parcela {parcela.upper()}"
    df_filtrado = df[(df["Parcela"] == parcela) & (df["Fecha"].dt.year == anio)]
    if df_filtrado.empty:
        return f"No se encontraron datos para la parcela {parcela} en el año {anio}."
    total = df_filtrado["Rentabilidad"].sum()
    return f"La rentabilidad total de la parcela {parcela} en {anio} fue de ${total:.2f}"

def porcentaje_costos_por_fase() -> str:
    df = pd.read_csv("docs/datos_agricolas_3000_registros.csv")
    costos_por_fase = df.groupby("Fase Productiva")["Costo Total"].sum()
    total_costos = costos_por_fase.sum()
    porcentajes = (costos_por_fase / total_costos * 100).round(2)
    resultado = "📊 Análisis porcentual de los costos por fase productiva:\n\n"
    for fase, porcentaje in porcentajes.items():
        resultado += f"• {fase}: {porcentaje:.2f}% del total de costos\n"
    return resultado or "No se pudo calcular un resultado en este momento."

def analisis_demanda_mercado() -> str:
    df = pd.read_csv("docs/datos_agricolas_3000_registros.csv")
    df = df.dropna(subset=["Demanda de Mercado", "Rentabilidad", "Precio por kg", "Cantidad (kg)"])
    resumen = df.groupby("Demanda de Mercado").agg({
        "Rentabilidad": "mean",
        "Precio por kg": "mean",
        "Cantidad (kg)": "sum"
    }).round(2)
    resultado = "📈 Análisis de la demanda de mercado:\n\n"
    for demanda, row in resumen.iterrows():
        resultado += (
            f"• Demanda **{demanda}**:\n"
            f"   - Rentabilidad promedio: ${row['Rentabilidad']:.2f}\n"
            f"   - Precio promedio por kg: ${row['Precio por kg']:.2f}\n"
            f"   - Volumen total: {int(row['Cantidad (kg)'])} kg\n\n"
        )
    return resultado or "No se pudo calcular un resultado en este momento."

def consultar_datos_agricolas(query: str) -> str:
    result = qa_chain.invoke({"query": query})
    if not result["result"].strip():
        return "No se encontró información relevante en el contexto para responder la pregunta."
    return result["result"]

tools = [
    StructuredTool.from_function(
        func=rentabilidad_por_parcela,
        name="rentabilidad_por_parcela",
        description="Calcula la rentabilidad total de una parcela para un año dado."
    ),
    Tool.from_function(
        func=porcentaje_costos_por_fase,
        name="porcentaje_costos_por_fase",
        description="Analiza los costos porcentuales por fase productiva."
    ),
    Tool.from_function(
        func=analisis_demanda_mercado,
        name="analisis_demanda_mercado",
        description="Analiza demanda de mercado vs rentabilidad, precio y volumen."
    ),
    Tool.from_function(
        func=consultar_datos_agricolas,
        name="consultar_datos_agricolas",
        description="Consulta el contexto vectorizado para responder preguntas generales."
    )
]

# ---------------------------------------------
# 5. Crear agente con prompt personalizado
# ---------------------------------------------
custom_markdown_template =  """
# Asistente de Rentabilidad Agrícola (RAG + Tools)

Eres un **asistente experto en análisis financiero agrícola** que puede usar herramientas para responder preguntas de forma precisa y fundamentada.

---

## Herramientas disponibles

- **`rentabilidad_por_parcela(parcela: str, anio: int)`**  
    Úsala para responder preguntas sobre la rentabilidad de una parcela en un año específico.  
    *Ejemplo:* "¿Cuál fue la rentabilidad de la Parcela A en 2023?"

- **`consultar_datos_agricolas(pregunta: str)`**  
    Úsala para obtener contexto general sobre clima, observaciones, demanda del mercado, etc., desde los documentos vectorizados.  
    *Ejemplo:* "¿Qué condiciones de mercado tuvo el maíz en invierno?"

---

## Reglas de comportamiento

- Siempre usa la Tool adecuada en lugar de intentar adivinar cifras.
- Sé claro, profesional y útil en tus respuestas.
- Si no hay datos suficientes, explica el motivo de forma transparente.
- No inventes datos numéricos ni te repitas.
- No des información especulativa si puedes llamar una herramienta.

---

## Objetivo

Tu propósito es ayudar a los usuarios a **tomar decisiones informadas** sobre la rentabilidad de sus cultivos, parcelas y líneas productivas.

{input}
"""

# ✅ Creamos el prompt con las variables necesarias
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content=custom_markdown_template),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])




llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    google_api_key=gemini_api_secret.get_secret_value(),
    temperature=0.2
)

# ✅ Creamos el agente con este prompt
from langchain.agents import create_structured_chat_agent
agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=prompt)

# ✅ Creamos el ejecutor del agente
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# ---------------------------------------------
# 6. Ejecutar una pregunta
# ---------------------------------------------
respuesta = agent_executor.invoke({"input": "¿Cuál fue la rentabilidad de la parcela B en 2023?"})
print("🧠", respuesta["output"])

ValueError: Prompt missing required variables: {'tool_names', 'tools'}